In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


class TabularDataset(Dataset):
    def __init__(self, data, cat_cols=None, output_col=None):
 

        self.n = data.shape[0]

        if output_col:
            self.y = data[output_col].astype(np.float32).values.reshape(-1, 1)
        else:
            self.y =  np.zeros((self.n, 1))

        self.cat_cols = cat_cols if cat_cols else []
        self.cont_cols = [col for col in data.columns
                          if col not in self.cat_cols + [output_col]]

        if self.cont_cols:
            self.cont_X = data[self.cont_cols].astype(np.float32).values
        else:
            self.cont_X = np.zeros((self.n, 1))

        if self.cat_cols:
            self.cat_X = data[cat_cols].astype(np.int64).values
        else:
            self.cat_X =  np.zeros((self.n, 1))

    def __len__(self):
    
        return self.n

    def __getitem__(self, idx):
   
        return [self.y[idx], self.cont_X[idx], self.cat_X[idx]]


class FeedForwardNN(nn.Module):

    def __init__(self, emb_dims, no_of_cont, lin_layer_sizes,
               output_size, emb_dropout, lin_layer_dropouts):

    
        super().__init__()

        # Embedding layers
        self.emb_layers = nn.ModuleList([nn.Embedding(x, y)
                                         for x, y in emb_dims])

        no_of_embs = sum([y for x, y in emb_dims])
        self.no_of_embs = no_of_embs
        self.no_of_cont = no_of_cont

        # Linear Layers
        first_lin_layer = nn.Linear(self.no_of_embs + self.no_of_cont,
                                    lin_layer_sizes[0])

        self.lin_layers =\
         nn.ModuleList([first_lin_layer] +\
              [nn.Linear(lin_layer_sizes[i], lin_layer_sizes[i + 1])
               for i in range(len(lin_layer_sizes) - 1)])

        for lin_layer in self.lin_layers:
            nn.init.kaiming_normal_(lin_layer.weight.data)

        # Output Layer
        self.output_layer = nn.Linear(lin_layer_sizes[-1],
                                      output_size)
        nn.init.kaiming_normal_(self.output_layer.weight.data)

        # Batch Norm Layers
        self.first_bn_layer = nn.BatchNorm1d(self.no_of_cont)
        self.bn_layers = nn.ModuleList([nn.BatchNorm1d(size)
                                        for size in lin_layer_sizes])

        # Dropout Layers
        self.emb_dropout_layer = nn.Dropout(emb_dropout)
        self.droput_layers = nn.ModuleList([nn.Dropout(size)
                                      for size in lin_layer_dropouts])

    def forward(self, cont_data, cat_data):

        if self.no_of_embs != 0:
            x = [emb_layer(cat_data[:, i])
               for i,emb_layer in enumerate(self.emb_layers)]
            x = torch.cat(x, 1)
            x = self.emb_dropout_layer(x)

        if self.no_of_cont != 0:
            normalized_cont_data = self.first_bn_layer(cont_data)

            if self.no_of_embs != 0:
                x = torch.cat([x, normalized_cont_data], 1) 
            else:
                x = normalized_cont_data

        for lin_layer, dropout_layer, bn_layer in\
            zip(self.lin_layers, self.droput_layers, self.bn_layers):

            x = F.relu(lin_layer(x))
            x = bn_layer(x)
            x = dropout_layer(x)

        x = self.output_layer(x)

        return x

In [2]:
data = pd.read_csv('data.csv')

In [3]:
data['date_in'] = pd.to_datetime(data['date_in'])
data['year'] = data['date_in'].map(lambda x: x.strftime('%Y'))
data['month'] = data['date_in'].map(lambda x: x.strftime('%m'))
data['day'] = data['date_in'].map(lambda x: x.strftime('%d'))

In [4]:
dr = ['date_in','house_pk']

data = data.drop(dr,axis = 1)
data.head().T

,0,1,2,3,4
agency_id,90,90,90,90,90
price,532,588,588,588,588
dis_water_real,0.261,0.261,0.261,0.261,0.261
dis_shopping,3,3,3,3,3
no_bedrooms,3,3,3,3,3
max_persons,4,4,4,4,4
house_size,140,140,140,140,140
land_size,726,726,726,726,726
build_year,1953,1953,1953,1953,1953
renovation_year,2014,2014,2014,2014,2014


In [5]:
categorical_features = ['agency_id', 'apartment', 'indoor_pool', 'spa', 'internet', 'pets_allowed', 'water_view', 'fire_stove', 'year', 'month', 'day']
output_columns = 'price'

In [6]:
data.shape

(85195, 21)

In [7]:
label_encoders = {}
for cat_col in categorical_features:
        label_encoders[cat_col] = LabelEncoder()
        data[cat_col] = label_encoders[cat_col].fit_transform(data[cat_col])

In [8]:
dataset = TabularDataset(data=data, cat_cols=categorical_features, output_col=output_columns)

In [9]:
batchsize = 64
dataloader = DataLoader(dataset, batchsize, shuffle=True)

In [10]:
cat_dims = [int(data[col].nunique()) for col in categorical_features]
cat_dims

[4, 2, 2, 2, 2, 2, 2, 2, 4, 12, 31]

In [11]:
emb_dims = [(x, min(50, (x + 1) // 2)) for x in cat_dims]
emb_dims

[(4, 2),
 (2, 1),
 (2, 1),
 (2, 1),
 (2, 1),
 (2, 1),
 (2, 1),
 (2, 1),
 (4, 2),
 (12, 6),
 (31, 16)]

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
model = FeedForwardNN(emb_dims, no_of_cont=9, lin_layer_sizes=[50, 100],
                          output_size=1, emb_dropout=0.04,
                          lin_layer_dropouts=[0.001,0.01]).to(device)

In [14]:
print(model)

FeedForwardNN(
  (emb_layers): ModuleList(
    (0): Embedding(4, 2)
    (1): Embedding(2, 1)
    (2): Embedding(2, 1)
    (3): Embedding(2, 1)
    (4): Embedding(2, 1)
    (5): Embedding(2, 1)
    (6): Embedding(2, 1)
    (7): Embedding(2, 1)
    (8): Embedding(4, 2)
    (9): Embedding(12, 6)
    (10): Embedding(31, 16)
  )
  (lin_layers): ModuleList(
    (0): Linear(in_features=42, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=100, bias=True)
  )
  (output_layer): Linear(in_features=100, out_features=1, bias=True)
  (first_bn_layer): BatchNorm1d(9, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn_layers): ModuleList(
    (0): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (emb_dropout_layer): Dropout(p=0.04)
  (droput_layers): ModuleList(
    (0): Dropout(p=0.001)
    (1): Dropout(p=0.01)
  )
)


In [16]:
epochs = 10

criterion = nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(), lr = 0.1)

for epoch in range(epochs):
    
    for y, cont_x, cat_x in dataloader:
        
        optimizer.zero_grad()
        
        cont_x = cont_x.to(device)
        cat_x = cat_x.to(device)
        y = y.to(device)
        
        output = model(cont_x, cat_x)
        
        loss = criterion(output, y)
        
        print('loss: {:.3f}'.format(loss.item()))
        
        loss.backward()
        
        optimizer.step()

loss: 18309.361
loss: 7895.047
loss: 7851.592
loss: 13652.759
loss: 16055.763
loss: 7882.958
loss: 7595.965
loss: 5422.001
loss: 8200.142
loss: 7426.079
loss: 3670.101
loss: 13038.594
loss: 15645.165
loss: 4434.312
loss: 4660.741
loss: 6633.661
loss: 7316.751
loss: 18340.396
loss: 7183.881
loss: 10155.275
loss: 22520.248
loss: 9225.364
loss: 18493.816
loss: 8548.258
loss: 15638.517
loss: 4155.973
loss: 8057.932
loss: 5460.534
loss: 10817.095
loss: 9520.183
loss: 12804.557
loss: 7843.785
loss: 5264.334
loss: 6453.734
loss: 7497.205
loss: 8939.227
loss: 10483.805
loss: 12595.352
loss: 11415.723
loss: 4486.149
loss: 5200.047
loss: 14624.079
loss: 6789.899
loss: 13656.205
loss: 6680.385
loss: 14359.768
loss: 7603.309
loss: 5008.361
loss: 7640.855
loss: 7178.400
loss: 10296.512
loss: 8156.110
loss: 7231.420
loss: 9849.177
loss: 13776.159
loss: 7868.854
loss: 18713.777
loss: 7893.656
loss: 16892.631
loss: 10291.432
loss: 5345.668
loss: 7473.259
loss: 11655.735
loss: 15387.549
loss: 26372.373

loss: 5413.367
loss: 8188.225
loss: 12023.221
loss: 9519.037
loss: 9909.547
loss: 15548.117
loss: 18703.471
loss: 5733.439
loss: 6076.092
loss: 12965.887
loss: 10112.609
loss: 7049.687
loss: 8414.092
loss: 9974.653
loss: 25839.486
loss: 8750.832
loss: 5739.636
loss: 6345.593
loss: 16477.092
loss: 8657.303
loss: 10180.441
loss: 5985.662
loss: 8570.673
loss: 6790.029
loss: 13794.081
loss: 13733.187
loss: 7776.471
loss: 6139.757
loss: 4471.768
loss: 11133.623
loss: 5548.651
loss: 11196.991
loss: 4517.373
loss: 10057.725
loss: 13050.766
loss: 16227.771
loss: 12830.292
loss: 6312.290
loss: 12358.729
loss: 6903.354
loss: 9361.296
loss: 7662.110
loss: 6287.982
loss: 14725.954
loss: 11254.112
loss: 19692.656
loss: 6793.112
loss: 5213.673
loss: 9570.087
loss: 10760.285
loss: 12760.293
loss: 6635.409
loss: 6608.812
loss: 9357.760
loss: 11146.246
loss: 9967.362
loss: 10195.270
loss: 6155.422
loss: 9506.663
loss: 6025.448
loss: 7567.785
loss: 8423.938
loss: 4898.585
loss: 24934.666
loss: 3730.118


loss: 9210.009
loss: 6573.364
loss: 10846.052
loss: 12216.671
loss: 3964.316
loss: 6705.154
loss: 10064.792
loss: 8908.858
loss: 9272.446
loss: 8625.524
loss: 8409.837
loss: 3703.211
loss: 5467.447
loss: 8184.132
loss: 10265.396
loss: 8069.941
loss: 6885.955
loss: 5698.764
loss: 6523.189
loss: 8318.068
loss: 5387.186
loss: 4350.310
loss: 11141.049
loss: 6493.314
loss: 8267.373
loss: 11494.930
loss: 8268.370
loss: 7206.371
loss: 8360.392
loss: 5159.677
loss: 4517.051
loss: 8149.652
loss: 17918.684
loss: 4622.354
loss: 7256.233
loss: 8335.021
loss: 11659.159
loss: 6013.018
loss: 6681.482
loss: 7216.680
loss: 7934.200
loss: 8542.265
loss: 14405.327
loss: 8811.219
loss: 6873.848
loss: 8395.550
loss: 17223.684
loss: 8104.649
loss: 7673.952
loss: 9719.606
loss: 16030.933
loss: 4510.412
loss: 7944.862
loss: 13022.773
loss: 5202.088
loss: 5978.183
loss: 5662.967
loss: 8138.344
loss: 7241.199
loss: 5024.250
loss: 12404.296
loss: 6264.146
loss: 5636.249
loss: 7556.743
loss: 7246.251
loss: 13293.

loss: 6802.950
loss: 8992.533
loss: 5609.646
loss: 5389.411
loss: 12785.149
loss: 6921.204
loss: 12136.108
loss: 5560.561
loss: 8345.447
loss: 4574.942
loss: 11580.879
loss: 8299.840
loss: 11656.626
loss: 7779.661
loss: 5402.758
loss: 18495.299
loss: 4804.234
loss: 7219.738
loss: 3897.261
loss: 7558.486
loss: 8289.589
loss: 8151.880
loss: 10864.709
loss: 9785.225
loss: 9703.332
loss: 9376.555
loss: 3168.988
loss: 7336.631
loss: 5696.833
loss: 9090.741
loss: 8504.969
loss: 6788.141
loss: 5103.275
loss: 8059.219
loss: 31009.271
loss: 16590.941
loss: 4995.250
loss: 7456.910
loss: 5653.721
loss: 8978.431
loss: 11910.447
loss: 15269.317
loss: 10195.090
loss: 9518.731
loss: 4729.964
loss: 3770.988
loss: 6517.238
loss: 16348.188
loss: 7295.842
loss: 4768.967
loss: 5227.346
loss: 7262.023
loss: 3515.578
loss: 9084.060
loss: 5509.433
loss: 12007.595
loss: 6288.403
loss: 6393.799
loss: 5647.400
loss: 18539.857
loss: 8522.684
loss: 5568.564
loss: 6194.889
loss: 17837.354
loss: 7187.405
loss: 5392

loss: 7891.978
loss: 4163.458
loss: 6636.929
loss: 7375.614
loss: 8464.611
loss: 7180.347
loss: 3717.459
loss: 16501.025
loss: 10514.361
loss: 4360.653
loss: 6368.779
loss: 9349.521
loss: 7002.883
loss: 9352.499
loss: 8554.463
loss: 4113.062
loss: 5980.466
loss: 8865.275
loss: 7962.493
loss: 6116.258
loss: 20950.861
loss: 8360.218
loss: 6724.126
loss: 7548.567
loss: 6527.444
loss: 8367.383
loss: 23396.027
loss: 17808.891
loss: 8074.220
loss: 5193.200
loss: 8946.083
loss: 3857.035
loss: 19751.658
loss: 9664.577
loss: 10313.070
loss: 3444.352
loss: 4947.337
loss: 6273.838
loss: 7269.634
loss: 8818.051
loss: 6798.853
loss: 4979.353
loss: 10229.072
loss: 5844.421
loss: 11101.334
loss: 6981.974
loss: 10213.523
loss: 5851.825
loss: 6811.000
loss: 7865.855
loss: 9631.667
loss: 5355.044
loss: 9878.455
loss: 4183.918
loss: 8336.487
loss: 4760.323
loss: 9376.102
loss: 9784.184
loss: 6363.753
loss: 6188.664
loss: 7076.035
loss: 10044.192
loss: 12562.580
loss: 4357.129
loss: 6416.914
loss: 5408.73

...

loss: 5122.948
loss: 4500.294
loss: 10771.715
loss: 3211.102
loss: 13851.441
loss: 7123.155
loss: 6088.200
loss: 9902.128
loss: 5224.314
loss: 11811.721
loss: 15514.659
loss: 6136.983
loss: 8963.257
loss: 5340.807
loss: 6061.102
loss: 8600.807
loss: 5920.816
loss: 7807.734
loss: 10673.887
loss: 7043.316
loss: 3980.301
loss: 4172.039
loss: 5078.532
loss: 6890.981
loss: 7640.120
loss: 7870.441
loss: 8667.425
loss: 10316.118
loss: 2686.431
loss: 4380.756
loss: 6565.758
loss: 6860.195
loss: 4746.944
loss: 3481.869
loss: 5195.115
loss: 3797.220
loss: 2834.583
loss: 11408.271
loss: 5681.482
loss: 8818.120
loss: 2609.073
loss: 8551.351
loss: 5449.313
loss: 7800.932
loss: 2882.877
loss: 10403.435
loss: 4652.749
loss: 3207.268
loss: 7952.030
loss: 6082.618
loss: 5500.639
loss: 14679.453
loss: 2184.389
loss: 4304.427
loss: 5231.753
loss: 6569.549
loss: 7785.479
loss: 6160.150
loss: 6463.300
loss: 11410.213
loss: 15249.423
loss: 5019.733
loss: 4778.230
loss: 15632.821
loss: 8477.962
loss: 15194.1

loss: 5218.677
loss: 2931.434
loss: 4264.421
loss: 11518.237
loss: 13070.623
loss: 6036.379
loss: 5991.885
loss: 9156.269
loss: 4645.139
loss: 6156.296
loss: 4443.152
loss: 6461.542
loss: 6451.820
loss: 5901.102
loss: 6351.407
loss: 6398.861
loss: 4989.018
loss: 5187.355
loss: 7552.469
loss: 11579.645
loss: 4984.697
loss: 12660.573
loss: 5659.828
loss: 2111.934
loss: 3597.893
loss: 3878.827
loss: 4117.839
loss: 4223.353
loss: 6969.977
loss: 5784.369
loss: 5734.593
loss: 4636.690
loss: 5927.245
loss: 5396.082
loss: 5726.988
loss: 8310.371
loss: 6715.543
loss: 5158.106
loss: 4494.061
loss: 5638.668
loss: 8129.258
loss: 5065.570
loss: 4906.312
loss: 10285.478
loss: 6889.249
loss: 8498.151
loss: 4217.355
loss: 10611.699
loss: 9842.938
loss: 4345.884
loss: 4943.517
loss: 19088.898
loss: 4731.028
loss: 5665.747
loss: 10268.372
loss: 5026.043
loss: 7031.665
loss: 4803.000
loss: 13032.301
loss: 8098.383
loss: 17046.738
loss: 5874.277
loss: 8895.667
loss: 4464.239
loss: 3867.596
loss: 3031.911


loss: 4072.206
loss: 5255.314
loss: 6489.560
loss: 14691.962
loss: 11736.651
loss: 4416.142
loss: 8821.456
loss: 10324.577
loss: 8160.504
loss: 5842.382
loss: 2628.104
loss: 5734.119
loss: 5337.990
loss: 22260.832
loss: 6775.293
loss: 3657.580
loss: 7270.458
loss: 8804.541
loss: 7439.832
loss: 6003.955
loss: 5571.238
loss: 4838.266
loss: 4830.504
loss: 8509.682
loss: 10783.163
loss: 10766.774
loss: 9309.865
loss: 9405.050
loss: 18405.139
loss: 8447.522
loss: 4333.414
loss: 9842.633
loss: 5121.683
loss: 9438.530
loss: 5774.017
loss: 4937.275
loss: 8026.488
loss: 7647.943
loss: 6487.126
loss: 5164.770
loss: 6134.905
loss: 17972.707
loss: 25518.334
loss: 4535.497
loss: 4727.639
loss: 9579.060
loss: 5811.665
loss: 7363.539
loss: 14950.243
loss: 3236.826
loss: 6782.617
loss: 6646.716
loss: 5137.568
loss: 20978.508
loss: 6086.974
loss: 5492.764
loss: 10443.326
loss: 10390.321
loss: 5475.983
loss: 10615.029
loss: 2776.287
loss: 9313.671
loss: 5484.056
loss: 7027.984
loss: 6022.547
loss: 22902

loss: 8883.358
loss: 10306.789
loss: 5190.660
loss: 9779.939
loss: 19714.186
loss: 5166.651
loss: 8351.731
loss: 6882.059
loss: 4965.752
loss: 7614.675
loss: 8264.396
loss: 5941.208
loss: 5813.142
loss: 11162.867
loss: 10517.163
loss: 9128.042
loss: 6964.478
loss: 13453.571
loss: 6651.762
loss: 22222.799
loss: 8477.306
loss: 6580.600
loss: 5863.432
loss: 4477.746
loss: 6351.677
loss: 4380.705
loss: 3929.235
loss: 8001.164
loss: 3800.719
loss: 4905.586
loss: 8102.373
loss: 8287.267
loss: 4636.147
loss: 17286.107
loss: 4923.409
loss: 7100.488
loss: 5179.328
loss: 6452.351
loss: 4742.624
loss: 5405.806
loss: 19344.754
loss: 12607.869
loss: 7828.512
loss: 12144.187
loss: 9266.694
loss: 10524.739
loss: 15593.385
loss: 14024.566
loss: 15401.129
loss: 5760.680
loss: 10821.278
loss: 13393.681
loss: 4336.901
loss: 8483.543
loss: 10181.143
loss: 5697.754
loss: 5162.665
loss: 6731.643
loss: 7815.349
loss: 6541.143
loss: 10985.321
loss: 11126.361
loss: 13707.070
loss: 6816.746
loss: 5510.182
loss:

loss: 5925.488
loss: 3132.760
loss: 7624.069
loss: 7541.684
loss: 2345.010
loss: 6608.231
loss: 4859.209
loss: 3064.711
loss: 5300.625
loss: 5627.541
loss: 3069.316
loss: 8560.319
loss: 4204.781
loss: 5580.418
loss: 5725.853
loss: 16204.313
loss: 5184.602
loss: 5776.963
loss: 11594.022
loss: 3844.511
loss: 8469.410
loss: 6276.118
loss: 4538.414
loss: 6470.261
loss: 7385.669
loss: 4884.150
loss: 6255.497
loss: 3506.840
loss: 6986.473
loss: 6761.898
loss: 6201.917
loss: 4432.562
loss: 3697.273
loss: 5608.521
loss: 2879.634
loss: 5941.331
loss: 9825.178
loss: 3569.976
loss: 10714.474
loss: 4650.051
loss: 2515.053
loss: 8059.037
loss: 10673.062
loss: 4012.920
loss: 7939.017
loss: 12988.067
loss: 6395.455
loss: 5405.487
loss: 12298.297
loss: 9246.019
loss: 19192.996
loss: 6569.842
loss: 10454.281
loss: 6446.746
loss: 7473.250
loss: 5285.636
loss: 5692.032
loss: 5659.498
loss: 6916.628
loss: 8650.277
loss: 5039.778
loss: 4712.637
loss: 14774.997
loss: 7106.074
loss: 5883.809
loss: 6512.208
l